# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

In [34]:
!kubectl label nodes apiserver-crd role=locust --overwrite

node "apiserver-crd" labeled


## Start seldon-core

In [21]:
!helm install ../../helm-charts/seldon-core --name seldon-core \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --set cluster_manager.image.tag=0.3-SNAPSHOT \
    --set apife.image.tag=0.1-SNAPSHOT \
    --set engine.image.tag=0.2-SNAPSHOT

NAME:   seldon-core
LAST DEPLOYED: Fri Dec  1 18:46:21 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                       DATA  AGE
alertmanager-server-conf   1     9s
grafana-import-dashboards  5     9s
prometheus-rules           4     9s
prometheus-server-conf     1     9s

==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           9s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          9s
seldon-apiserver         1        1        1           0          9s
seldon-cluster-manager   1        1        1           0          9s
grafana-prom-deployment  1        1        1           0          9s
kafka                    1        1        1           0          9s
prometheus-deployment    1        1        1           0          9s
redis                    1        1        1           0     

In [22]:
!helm status seldon-core

LAST DEPLOYED: Fri Dec  1 18:46:21 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                       DATA  AGE
alertmanager-server-conf   1     17s
grafana-import-dashboards  5     17s
prometheus-rules           4     17s
prometheus-server-conf     1     17s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  17s

==> v1/Service
NAME                      TYPE       CLUSTER-IP  EXTERNAL-IP  PORT(S)                     AGE
alertmanager              ClusterIP  10.0.0.144  <none>       80/TCP                      17s
seldon-apiserver          NodePort   10.0.0.14   <none>       8080:30032/TCP              17s
grafana-prom              NodePort   10.0.0.195  <none>       80:30034/TCP                17s
kafka                     NodePort   10.0.0.76   <none>       9092:30010/TCP              17s
prometheus-node-exporter  ClusterIP  None        <none>       9100/TCP                  

# Integrating with Kubernetes API

In [23]:
!cat resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.6",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
         

## Create Seldon Deployment

In [24]:
!kubectl apply -f resources/model.json

seldondeployment "seldon-deployment-example" created


In [25]:
!kubectl get seldondeployments

NAME                        AGE
seldon-deployment-example   1s


In [26]:
!kubectl describe seldondeployments seldon-deployment-example 

Name:         seldon-deployment-example
Namespace:    default
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"SeldonDeployment","metadata":{"annotations":{},"labels":{"app":"seldon"},"name":"seldon-depl...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         SeldonDeployment
Metadata:
  Cluster Name:                   
  Creation Timestamp:             2017-12-01T18:47:08Z
  Deletion Grace Period Seconds:  <nil>
  Deletion Timestamp:             <nil>
  Initializers:                   <nil>
  Resource Version:               72521
  Self Link:                      /apis/machinelearning.seldon.io/v1alpha1/namespaces/default/seldondeployments/seldon-deployment-example
  UID:                            087ad66f-d6c8-11e7-af35-080027b241c6
Spec:
  Annotations:
    Project _ Name:  FX Market Prediction
  Name:              test-deployment
  Oauth _ Key:       oauth-key
  Oauth _ Secre

In [27]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1]]]

## Get predictions

In [28]:
%%bash
SERVER=192.168.99.100:30032
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"data":{"names":["a","b"],"tensor":{"shape":[1,2],"values":[0,0]}}}'

{
  "meta": {
    "puid": "gkcs3nlbecv5fa9ojpajub8fcr",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.05133579311531625]
    }
  }
}

## Update deployment with canary

In [29]:
!cat resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.6",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
         

In [30]:
!kubectl apply -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" configured


In [31]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1] map[replicas:1 replicasAvailable:1 name:test-deployment-fx-market-predictor-canary]]]

In [32]:
%%bash
SERVER=192.168.99.100:30032
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"data":{"names":["a","b"],"tensor":{"shape":[1,2],"values":[0,0]}}}'

{
  "meta": {
    "puid": "80enhjvh07uso0pcuaa940t2e8",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.05133579311531625]
    }
  }
}

## Load test

In [33]:
!helm install ../../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret 

NAME:   loadtest
LAST DEPLOYED: Fri Dec  1 18:52:25 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Pod(related)
NAME                   READY  STATUS   RESTARTS  AGE
locust-slave-1-4rb42   0/1    Pending  0         0s
locust-master-1-tzhl6  0/1    Pending  0         0s

==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP  EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.0.0.2    <none>       5557:32519/TCP,5558:30586/TCP,8089:31118/TCP  0s




# Tear down

In [35]:
!helm delete loadtest --purge

release "loadtest" deleted


In [36]:
!kubectl delete -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" deleted


In [37]:
!helm delete seldon-core --purge

release "seldon-core" deleted
